In [82]:
import numpy as np
import pandas as pd
from ase import Atoms
from ase.io import read, write

# Define function to extract atomic masses from LAMMPS input file
def extract_masses(file_path):
    start_keyword = "Masses"
    end_keyword = "Atoms"

    with open(file_path, 'r') as file:
        lines = file.readlines()

    start_index, end_index = None, None
    for i, line in enumerate(lines):
        if start_keyword in line:
            start_index = i + 1
        elif end_keyword in line and start_index is not None:
            end_index = i  
            break

    if start_index is not None and end_index is not None:
        extracted_data = [line.strip() for line in lines[start_index:end_index] if line.strip()]
        return extracted_data
    else:
        return None

# Example file path (replace with actual file)
file_path = "data.sim"

# Extract masses and store in DataFrame
masses_data = extract_masses(file_path)
if masses_data:
    data = []
    for line in masses_data:
        parts = line.split()
        if len(parts) >= 2:
            atom_type = parts[0]
            mass = parts[1]
            description = " ".join(parts[2:]) if len(parts) > 2 else ""
            data.append({"Atom Type": atom_type, "Mass": mass, "Description": description})

    df = pd.DataFrame(data)
else:
    print("No data found between 'Masses' and 'Atoms'.")
    df = pd.DataFrame()  # Create an empty DataFrame to prevent errors

# Read LAMMPS coordinate data
input_lammps_name = "dump.Pt_ads_water_prod.lammpstrj"  # Replace with actual filename

id_list, x, y, z = [], [], [], []
Length = []

with open(input_lammps_name) as f:
    for line in f:
        p = line.split()
        if len(p) == 5:
            id_list.append(p[1])
            x.append(np.float32(p[2]))
            y.append(np.float32(p[3]))
            z.append(np.float32(p[4]))
        elif len(p) == 2 and p[0] != 'ITEM:':
            Length.append(float(p[1]))

# Extract lattice constants
lx, ly, lz = [], [], []
for i in range(0, len(Length), 3):
    lx.append(Length[i])
    ly.append(Length[i+1])
    lz.append(Length[i+2])

Lx = lx[0] if all(v == lx[0] for v in lx) else None
Ly = ly[0] if all(v == ly[0] for v in ly) else None
Lz = lz[0] if all(v == lz[0] for v in lz) else None

# Extract atom types safely
atoms = []
if not df.empty and "Description" in df.columns:
    for desc in df["Description"]:
        parts = desc.split("#")
        if len(parts) > 1:
            atoms.append(parts[1].strip())
        else:
            atoms.append(desc.strip())  # Fallback to full description if '#' is missing

# Map atom names
ATOMS1 = []
for atom in atoms:
    if atom == 'Ow':
        ATOMS1.append('OW')
    elif atom == 'Hw':
        ATOMS1.extend(['HW2', 'HW3'])  # Add both hydrogens directly
    elif atom == 'Mw':
        ATOMS1.append('MW4')
    else:
        ATOMS1.append(atom)

# Generate XYZ frames
Natoms = 4 * NWaterMolecule + Natoms_ads + Natoms_surface  # Define these values before running
Latice_constant = [Lx, Ly, Lz]
Latice_matrix = [[Lx, 0, 0], [0, Ly, 0], [0, 0, Lz]]

ID = [atoms[int(idx)-1] for idx in id_list] if id_list else []

for i in range(int(len(ID) / Natoms)):
    IDD = ID[i*Natoms:i*Natoms+Natoms]
    X = x[i*Natoms:i*Natoms+Natoms]
    Y = y[i*Natoms:i*Natoms+Natoms]
    Z = z[i*Natoms:i*Natoms+Natoms]

    filename = f"frame{i+1}.xyz"
    with open(filename, 'w') as file:
        file.write(str(Natoms) + '\n')
        file.write('system at Frame = ' + str(i+1) + '\n')
        for j in range(len(IDD)):
            file.write(f"{IDD[j]:<5}{np.round(X[j]*Latice_constant[0],3):<7}{np.round(Y[j]*Latice_constant[1],3):<7}{np.round(Z[j]*Latice_constant[2],3):<7}\n")


In [86]:

numFrames = len(ID) // Natoms
for t in range(numFrames):
    with open('frame'+str(t+1)+'.xyz') as f:
        lines = f.readlines()[2:]  

    data_p = []
    for line in lines:
        p = line.split()
        if len(p) == 4:
            symbol, x, y, z = p[0], float(p[1]), float(p[2]), float(p[3])
            data_p.append({"Atom Type": symbol, "X": x, "Y": y, "Z": z})

    df1 = pd.DataFrame(data_p)
    df2 = df1[df1['Atom Type'] != 'Mw'].reset_index(drop=True)
    df2["Atom Type"] = df2["Atom Type"].replace({"Ow": "O", "Hw": "H"})
    atom_order = ["Pt", "O", "H", "C"]
    df2["Atom Type"] = pd.Categorical(df2["Atom Type"], categories=atom_order, ordered=True)
    df_sorted = df2.sort_values(by="Atom Type").reset_index(drop=True)


    frame_number = t+1  # Modify if needed
    filename = f"new-sorted{frame_number}.xyz"
    with open(filename, 'w') as file:
        file.write(f"{len(df_sorted)}\n")  # Write number of atoms
        file.write(f"system at Frame = {frame_number}\n")  # Comment line
        for j in range(len(df_sorted)):
            file.write(f"{df_sorted['Atom Type'][j]:<3} {df_sorted['X'][j]:>15.6f} {df_sorted['Y'][j]:>15.6f} {df_sorted['Z'][j]:>15.6f}\n")

    atoms_xyz_to_gro = read(filename)
    gro_filename = f"new-sorted{frame_number}.gro"
    write(gro_filename, atoms_xyz_to_gro)
    atoms_xyz_to_gro.set_cell([Latice_constant[0], Latice_constant[1], Latice_constant[2]])  # Replace with actual box dimensions
    atoms_xyz_to_gro.set_pbc(True)  # Enables periodic boundary conditions
    write(gro_filename, atoms_xyz_to_gro)

    atoms_gro_to_vasp = read(gro_filename)
    vasp_filename = f"FRAME{frame_number}.vasp"
    write(vasp_filename, atoms_gro_to_vasp)


DEPRECATION: Loading egg at /Users/alirezahosseini/anaconda3/lib/python3.11/site-packages/WHAM-1.0-py3.11-macosx-11.1-arm64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [67]:
ID

['Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'Pt',
 'O',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'H',
 'H',
 'H',
 'H',
 'H',
 'H',
 'Ow',
 'Hw',
 'Hw',
 'Mw',
 'Ow',
 'Hw',
 'Hw',
 'Mw',
 'Ow',
 'Hw',
 'Hw',
 'Mw',
 'Ow',
 'Hw',
 'Hw',
 'Mw',
 'Ow',
 'Hw',
 'Hw',
 'Mw',
 'Ow',
 'Hw',
 'Hw',
 'Mw',
 'Ow',
 'Hw',
 'Hw',
 'Mw',
 'Ow',
 'Hw',
 'Hw',
 'Mw',
 'Ow',
 'Hw',
 'Hw',
 'Mw',
 'Ow',
 'Hw',
 'Hw',
 'Mw',
 'Ow',
 'Hw',
 'Hw',
 'Mw',
 'Ow',
 'Hw',
 'Hw',
 'Mw',
 'Ow',
 'Hw',
 'Hw',
 'Mw',
 'Ow',
 'Hw',
 'Hw',
 'Mw',
 'Ow',
 'Hw',
 'Hw',

In [85]:
numFrames

8